# Iris Flower Classification with Scikit-Learn and Hopsworks

In this notebook we will, 

1. Import libraries and connect to Hopsworks Feature Store
2. Load the iris Flower dataset
3. Create a feature group and upload to the feature store
4. Create a feature view from the feature group
5. Create a training dataset
6. Train a model using SkLearn
7. Save the trained model to Hopsworks
8. Launch a serving instance.
9. Model deployment in Hopsworks
10. Send a prediction request to the served model
11. Try out your Model Interactively with a Gradio UI 


In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
import numpy as np
import time
import json
import random
import hopsworks
import pandas as pd
from sklearn import preprocessing

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [2]:
iris_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/iris.csv")
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [3]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

We can save two feature groups (hive tables), one called `iris_features` that contains the iris features and the corresponding numeric label, and another feature group called `iris_labels_lookup` for converting the numeric iris label back to categorical.

**Note**: To be able to run the feature store code, you first have to enable the Feature Store Service in your project. To do this, go to the "Settings" tab in your project, select the feature store service and click "Save". 

In [4]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/2238


Connected. Call `.close()` to terminate connection gracefully.


In [5]:
iris_fg = fs.get_or_create_feature_group(name="iris",
                                         version=1,
                                         primary_key=["sepal_length","sepal_width","petal_length","petal_width"],
                                         description="Iris flower dataset")
iris_fg.insert(iris_df)

Uploading Dataframe: 0.00% |          | Rows 0/150 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/2238/jobs/named/iris_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7ff92bc10250>, None)

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

Feature views are used to read features for training and inference.
If the feature view already exists, get it. If not, an exception is thrown, and we create the feature view.

In [6]:
try:
    feature_view = fs.get_feature_view(name="iris", version=1)
except:
    # Feature Selection
    query = iris_fg.select_all()
    feature_view = fs.create_feature_view(name="iris",
                                      version=1,
                                      description="Read from Iris flower dataset",
                                      labels=["variety"],
                                      query=query)

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_train_validation_test_split()` method.

* `X_train` is the train set features
* `X_val` is the validation set features
* `X_test` is the test set features
* `Y_train` is the train set labels
* `Y_val` is the validation set labels
* `Y_test` is the test set labels

In [7]:
td_version, td_job = feature_view.create_train_validation_test_split(
    description = 'iris tutorial',
    data_format = 'csv',
    validation_size = 0.2,
    test_size = 0.1,
    write_options = {'wait_for_job': True},
    coalesce = True,
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/2238/jobs/named/iris_1_4_create_fv_td_18102022081050/executions


In [21]:
X_train, X_val, X_test, y_train, y_val, y_test = feature_view.get_train_validation_test_split(td_version)

In [22]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
0,4.300000,3.000000,1.100000,0.100000
1,4.400000,2.900000,1.400000,0.200000
2,4.400000,3.200000,1.300000,0.200000
3,4.593688,4.064348,1.786139,0.683110
4,4.600000,3.100000,1.500000,0.200000
...,...,...,...,...
111,7.700000,3.000000,6.100000,2.300000
112,7.700000,3.800000,6.700000,2.200000
113,7.742389,2.863211,5.029597,1.586937
114,7.900000,3.800000,6.400000,2.000000


## <span style="color:#ff5f27;"> 🧬 Modeling</span>

Train a KNN (k-nearest neighbors) model with Scikit-learn. Use a label encoder to map the categorical labels to numbers.

,sepal_length,sepal_width,petal_length,petal_width
0,-1.836910,-0.143036,-1.518564,-1.520211
1,-1.725797,-0.364826,-1.353532,-1.385516
2,-1.725797,0.300542,-1.408542,-1.385516
3,-1.510586,2.217571,-1.141113,-0.734789
4,-1.503572,0.078753,-1.298521,-1.385516
...,...,...,...,...
111,1.940924,-0.143036,1.231983,1.443084
112,1.940924,1.631277,1.562049,1.308388
113,1.988023,-0.446420,0.643144,0.482622
114,2.163149,1.631277,1.397016,1.038998


In [24]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_train_encoded = le.fit_transform(y_train['variety'])
y_test_encoded = le.transform(y_test['variety'])

ss = preprocessing.StandardScaler()
X_train = pd.DataFrame(ss.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

In [25]:
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train_encoded) 

KNeighborsClassifier(n_neighbors=4)

### Evalute model performance

Compute the MSE of the model.

In [26]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test_encoded, y_pred)

metrics = {
    "mse" : mse
}
print(metrics)

{'mse': 0.0}


## <span style="color:#ff5f27;">📝 Register model</span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Save the following pickled objects as .pkl files locally to a directory that will be uploaded later to the model registry:

 * the model object, **model** saved as iris_model.pkl
 * the label encoder object, **le** saved as iris_encoder.pkl, so that we can reconstruct categorical names 
    from the encoded predictions (numbers) 
    
The model input schema is the same set of features as in the *x_train* DataFrame.

The model output schema is the same label as in the *y_train* DataFrame.

Finally, lazily create the model that will be register, including all files (artifacts) in the directory (containing the pickled label encoder object and the pickled model object), the model's input/output schema, and a sample input row (**input_example**). The model registry is the **mr** object, and for our Scikit-Learn model, we create a model of type Python with **mr.python.create_model()**. For TensorFlow, there is *mr.tensorflow.create_model()*.

In [27]:
import joblib
import os

# The 'iris_model' directory will be saved to the model registry
model_dir="iris_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
    
joblib.dump(model, model_dir + '/knn_iris_model.pkl')
joblib.dump(le, model_dir + '/knn_iris_encoder.pkl')

['iris_model/knn_iris_encoder.pkl']

In [28]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [29]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

iris_model = mr.python.create_model(
    name="knn_iris_model", 
    metrics=metrics,
    model_schema=model_schema,
    input_example=X_train.sample(), 
    description="Iris Flower Predictor")

iris_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/2238/models/knn_iris_model/1


Model(name: 'knn_iris_model', version: 1)

### <span style="color:#ff5f27;">📎 Predictor script for Python models</span>


Scikit-learn models are deployed as Python models, in which case you need to provide a **Predict** class that implements the **predict** method. The **predict()** method invokes the model on the inputs and returns the prediction as a list.

The **init()** method is run when the predictor is loaded into memory, loading the model from the local directory it is materialized to, *ARTIFACT_FILES_PATH*.

The directive "%%writefile" writes out the cell before to the given Python file. We will use the **iris_predictor.py** file to create a deployment for our Scikit-Learn K-NN model. 

In [30]:
%%writefile predict_example.py

import joblib
import os

class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the local path to the model artifact files        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/knn_iris_model.pkl")


    def predict(self, inputs):
        """ Serves a prediction request from a trained model"""
        return self.model.predict(inputs).tolist()

Writing predict_example.py


## <a class="anchor" id="1.5_bullet" style="color:#ff5f27"> 🚀 Model Deployment</a>

Provide the predictor script because it is a Python model (Scikit-Learn)

In [31]:
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/417 elapsed<00:00 remaining<?

In [32]:
ms = project.get_model_serving()
try:
    deployment = ms.get_deployment("irisdeployed")
except:
    deployment = iris_model.deploy(name="irisdeployed",
                                   script_file=predictor_script_path,  
                                   model_server="PYTHON", 
                                   serving_tool="KSERVE")

print("Deployment: " + deployment.name)
deployment.describe()

Connected. Call `.close()` to terminate connection gracefully.
Deployment created, explore it at https://c.app.hopsworks.ai:443/p/2238/deployments/2067
Before making predictions, start the deployment by using `.start()`
Deployment: irisdeployed
{
    "artifact_version": 1,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2022-10-18T08:17:46.034Z",
    "creator": "Tomasz Miko\u0142ajczyk",
    "id": 2067,
    "inference_logging": "NONE",
    "model_name": "knn_iris_model",
    "model_path": "/Projects/serverlessmltm/Models/knn_iris_model",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "irisdeployed",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 0.5,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "memory": 32
        }
    },
    "requested_instances": 0,
    "serving_to

### The deployment has now been registered. However, to start it you need to run:

In [39]:
state = deployment.get_state()

if state.status != "Running":
    deployment.start()
    deployment.describe()
else:
    print("Deployment already running")

Deployment is already starting
{
    "artifact_version": 1,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2022-10-18T08:17:46.034Z",
    "creator": "Tomasz Miko\u0142ajczyk",
    "id": 2067,
    "inference_logging": "NONE",
    "model_name": "knn_iris_model",
    "model_path": "/Projects/serverlessmltm/Models/knn_iris_model",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "irisdeployed",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 0.5,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "memory": 32
        }
    },
    "requested_instances": 0,
    "serving_tool": "KSERVE"
}


## <span style='color:#ff5f27'>🔮 Predicting using deployment</span>

In [ ]:
test_data = list(iris_model.input_example)

data = {"instances" : [test_data]}
res = deployment.predict(data)
print(test_data)
#print(le.inverse_transform([res["predictions"][0]]))
print([res["predictions"][0]])

## <span style="color:#ff5f27;"> 👾 Try out your Model Interactively </span> 


We will build a user interface with Gradio to allow you to enter the 4 feature values (sepal length/width and petal length/width), producing a prediction of the type of iris flower.

First, we have to install the gradio library.

In [ ]:
!pip install gradio --quiet

### Run Gradio

Start the Gradio UI. Users enter the 4 feature values and a prediction is returned. We use the label encoder object to transform the number returned to the categorical value (stringified name of the Iris Flower).

In [ ]:
import gradio as gr


def iris(sl, sw, pl, pw):
    list_inputs = []
    list_inputs.append(sl)
    list_inputs.append(sw)
    list_inputs.append(pl)
    list_inputs.append(pw)
    data = {
        "instances": [list_inputs]
    }
    res = deployment.predict(data)
    # Convert the numerical representation of the label back to it's original iris flower name.
    return le.inverse_transform([res["predictions"][0]])[0]

demo = gr.Interface(
    fn=iris,
    title="Iris Flower Predictive Analytics",
    description="Experiment with sepal/petal lengths/widths to predict which flower it is.",
    allow_flagging="never",
    inputs=[
        gr.inputs.Number(default=1.0, label="sepal length (cm)"),
        gr.inputs.Number(default=1.0, label="sepal width (cm)"),
        gr.inputs.Number(default=1.0, label="petal length (cm)"),
        gr.inputs.Number(default=1.0, label="petal width (cm)"),
        ],
    outputs="text")

demo.launch(share=True)